In [ ]:
import os
from os.path import join, exists, basename, dirname, expanduser
from glob import glob
import shutil
import numpy as np
import pandas as pd
import pickle
from datetime import datetime
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import xarray as xa
import rioxarray as rxa
from rioxarray.merge import merge_arrays, merge_datasets
from shapely.geometry import box

In [ ]:
with open(expanduser('~/scratch/data/uavsar/image_fps'), 'rb') as f:
    image_fps = pickle.load(f)

image_fps = [f for f in image_fps if f['fp'].endswith('.unw.grd.tiff')]

In [ ]:
image_fps[0].keys()

In [ ]:
cor = rxa.open_rasterio(image_fps[0]['cor'])
tree_fp = expanduser('~/scratch/data/uavsar/trees/Forest_height_2019_NAM.tif')
trees = rxa.open_rasterio(tree_fp)

In [ ]:
tree_cl = trees.rio.clip_box(*cor.rio.bounds())
cor_re = cor.rio.reproject_match(tree_cl) 
tree_cl_ma = tree_cl.where(~np.isnan(cor_re), np.nan)

In [ ]:
new = xa.Dataset()
new['trees'] = tree_cl_ma
new['cor'] = cor_re
cor_1d = cor_re.data.ravel()
tree_1d = tree_cl_ma.data.ravel()
not_nan = (~np.isnan(tree_1d)) & (~np.isnan(cor_1d))
cor_n1d = cor_1d[not_nan]
tree_n1d = tree_1d[not_nan]
N = int(1e6)
length = len(tree_n1d)
idx = np.random.choice(length, N, replace = False)
res = pd.DataFrame()
for name in new.data_vars:
    res.loc[:,name] = new[name].data.ravel()[not_nan][idx]

In [ ]:
plt.hist2d(x = res.loc[res.trees < 20].trees, y = res.loc[res.trees < 20].cor, bins = [50,50], norm=mpl.colors.LogNorm())
plt.show()

In [ ]:
f, ax = plt.subplots(figsize=(7, 7))
x = res.trees[::100]
y = res.cor[::100]
sns.kdeplot(x = x, y = y, shade=True, ax=ax)

In [ ]:
tmp_dir = expanduser(f'~/uavsar/results/coh_amp_inc/tmp/')
print('Combining tmp dictionaries.')
super_x = []
from tqdm import tqdm
for fp in tqdm(glob(join(tmp_dir, '*'))):
    with open(fp, 'rb') as f:
        tmp = pickle.load(f)
    super_x.append(tmp)
# res = pd.concat(super_x)

In [ ]:
res = pd.concat(super_x)

In [ ]:
plt.hist2d(np.rad2deg(np.isfinite(res.inc).values), np.isfinite(res.cor).values, bins = [100,100]) #norm=mpl.colors.LogNorm()

In [ ]:
f, ax = plt.subplots(figsize=(7, 7))
x = np.rad2deg(res.inc[::100].values)
y = res.cor[::100].values
sns.kdeplot(x = x, y = y, shade=True, ax=ax)